# Merging CNF, BLIF, N-queens
tested benchmarks:
- CNF: uf20-01 -- uf20-1000
- BLIF: C432, C499, C880, C1355, C1908
- N-queens: N in {4,5,6,7,8}

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams.update({
    # "mathtext.default": "regular",  # Use mathtext (math fonts) instead of LaTeX
    # "font.family": "serif",  # Use serif font (like Computer Modern)
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Computer Modern Roman"],
    "text.latex.preamble": r"\usepackage{amsmath}"
})

In [ ]:
dimacs_nodecount_csv = "./csv/dimacs20-nc.csv"
blif_nodecount_csv = "./csv/blif-nc.csv"
nqueens_nodecount_csv = "./csv/nqueens-nc.csv"

In [ ]:
dim_df = pd.read_csv(dimacs_nodecount_csv)
dim_df.rename(columns={'esr': 'esrbdd'}, inplace=True)

blif_df = pd.read_csv(blif_nodecount_csv)
blif_df.rename(columns={'esr': 'esrbdd'}, inplace=True)

nq_df = pd.read_csv(nqueens_nodecount_csv)

In [ ]:
all_cols = ['benchmark', 'init', 'unfold', 'normal', 'bdd', 'zbdd', 'tbdd', 'cbdd', 'czdd', 'esrbdd', 'abdd']
dim_df = dim_df.reindex(columns=all_cols, fill_value=0)
blif_df = blif_df.reindex(columns=all_cols, fill_value=0)
nq_df = nq_df.reindex(columns=all_cols, fill_value=0)

node_counts = pd.concat([blif_df, dim_df, nq_df], ignore_index=True)

# node_counts.to_csv("all_benchmarks_raw.csv", index=False)

In [ ]:
def classify_benchmark(name: str):
    if 'iscas' in name:
        return 'blif'
    elif name.startswith('uf20'):
        return 'cnf'
    elif name.startswith('queens'):
        return 'nqueens'

node_counts['type'] = node_counts['benchmark'].apply(classify_benchmark)

marker_map = {
    'blif': '.',
    'cnf': 'x',
    'nqueens': '^'
}

TEXTSIZE=20

targets = ['bdd', 'zbdd', 'tbdd', 'cbdd', 'czdd', 'esrbdd']
for t in targets:
    plt.figure(figsize=(6,6))
    for btype, marker in marker_map.items():
        subset = node_counts[node_counts['type'] == btype]
        plt.scatter(
            subset[t], subset['abdd'],
            label=btype,
            marker=marker,
            s=100,
            linewidths=1
            # color='black'
        )
    plt.tick_params(axis='x', labelsize=TEXTSIZE)
    plt.tick_params(axis='y', labelsize=TEXTSIZE)
    plt.xlabel(f"{t.upper()} node count", fontsize=TEXTSIZE)
    plt.ylabel(f"ABDD node count", fontsize=TEXTSIZE)
    xmin, xmax, ymin, ymax = 1e1, 1e5, 1e1, 1e5
    plt.xscale("log")
    plt.yscale("log")
    plt.legend(fontsize=TEXTSIZE)
    plt.grid(True, which='major', linestyle='--', linewidth=0.5)
    lims = [max(xmin, ymin), min(xmax, ymax)]
    plt.plot(lims, lims, 'k--', linewidth=1)
    plt.tight_layout()

    plt.savefig(f"./figures/all-bench-vs-{t.lower()}.pdf", format='pdf', dpi=300, facecolor='white', edgecolor='r', bbox_inches='tight', pad_inches=0.02)
    plt.show()